In [3]:
%matplotlib inline
%config IPCompleter.greedy=True 
import numpy as np #библиотека массивов и матриц
import matplotlib.pyplot as plt #низкоуровневая библиотека отображения графиков
import seaborn as sns # высокоуровневая библиотека визуализации
import pandas as pd # библиотека работы с графиками
import re #библиотека для работы с текстами
import pickle
import logging

# библиотеки для машинного обучения 

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from gensim.models import Word2Vec

C:\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


# подключаем библиотеку keras

In [5]:
from keras import backend as K
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import optimizers
from keras.layers import Dense, concatenate, Activation, Dropout
from keras.models import Model
from keras.models import load_model
from keras.layers.convolutional import Conv1D
from keras.layers.pooling import GlobalMaxPooling1D
from keras.layers import Input
from keras.layers.embeddings import Embedding
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


# библиотека для пост обработки

# размеры входной матрицы модели

In [8]:
SENTENCE_LENGTH = 26 # Высота матрицы (максимальное количество слов в предложении)
NUM = 100000         # размер словаря

# метод отчистки текста

In [1]:
def ClearText(text):
    text = text.lower().replace("ё", "е")
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', text)
    text = re.sub('@[^\s]+', 'USER', text)
    text = re.sub('[^a-zA-Zа-яА-Я1-9]+', ' ', text)
    text = re.sub(' +', ' ', text)
    return text.strip()

сказать видеть наступать грабли разочаровывать натравлять
Wall time: 1.12 s


In [2]:
n = ['id', 'date', 'name', 'text', 'typr', 'rep', 'rtw', 'faw', 'stcount', 'foll', 'frien', 'listcount']
data_positive = pd.read_csv('../data/positive.csv', sep=';', error_bad_lines=False, names=n, usecols=['text'])
data_negative = pd.read_csv('../data/negative.csv', sep=';', error_bad_lines=False, names=n, usecols=['text'])

NameError: name 'pd' is not defined

Wall time: 0 ns


In [6]:
sample_size = min(data_positive.shape[0], data_negative.shape[0])
raw_data = np.concatenate((data_positive['text'].values[:sample_size],
                           data_negative['text'].values[:sample_size]), axis=0)
labels = [1] * sample_size + [0] * sample_size

['школоть поверять самый d общество профилирующий предмет тип', 'весь таки немного похожий мальчик весь равный хороший d', 'rt идиотка испугаться', 'rt угол сидеть погибать голод 2 порция взять хотя жрать хотеть dd']


In [22]:
data = [ClearText(t) for t in raw_data]

In [23]:
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=1)

In [27]:
with open('../data/x_test.pickle', 'wb') as handle:
    pickle.dump(x_test, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [28]:
with open('../data/y_test.pickle', 'wb') as handle:
    pickle.dump(y_test, handle, protocol=pickle.HIGHEST_PROTOCOL)

# токенизатор предложений

In [9]:
tokenizer = Tokenizer(num_words=NUM)
tokenizer.fit_on_texts(x_train)

NameError: name 'x_train' is not defined

# сохраняем токенизатор

In [ ]:
with open('cnn/tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# загружаем токенизатор

In [10]:
with open('model/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

# методы вычисления ошибки и точности (метрики)

In [11]:
def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

# метод для преобразования предложения

In [12]:
def GetSequences(tokenizer, x):
    sequences = tokenizer.texts_to_sequences(x)
    return pad_sequences(sequences, maxlen=SENTENCE_LENGTH)

# метод для выставления оценки

In [13]:
def GetAnalis(text):
    clear = ClearText(text)
    test = GetSequences(tokenizer, [clear])
    predicted = model.predict(test)
    print('предложение: {}'.format(text))
    print('предобработанное предложение: {}' .format(clear))
    print('Результат: {} оценка: ' .format(predicted),
          'положительная' if predicted > 0.5 else 'негативная')

# метод визуализации

In [14]:
plt.style.use('ggplot')


def plot_metrix(ax, x1, x2, title):
    ax.plot(range(1, len(x1) + 1), x1, label='train')
    ax.plot(range(1, len(x2) + 1), x2, label='val')
    ax.set_ylabel(title)
    ax.set_xlabel('Epoch')
    ax.legend()
    ax.margins(0)


def plot_history(history):
    fig, axes = plt.subplots(ncols=2, nrows=2, figsize=(16, 9))
    ax1, ax2, ax3, ax4 = axes.ravel()
    plot_metrix(ax1, history.history['precision'], history.history['val_precision'], 'Precision')
    plot_metrix(ax2, history.history['recall'], history.history['val_recall'], 'Recall')
    plot_metrix(ax3, history.history['f1'], history.history['val_f1'], "$F_1$")
    plot_metrix(ax4, history.history['loss'], history.history['val_loss'], 'Loss')
    plt.show()

# преобразуем предложения.

In [30]:
x_train_seq = GetSequences(tokenizer, x_train)
x_test_seq = GetSequences(tokenizer, x_test)

In [31]:
with open('../data/x_test.pickle', 'wb') as handle:
    pickle.dump(x_test_seq, handle, protocol=pickle.HIGHEST_PROTOCOL)

# модель word2vec

In [15]:
w2v_model = Word2Vec.load("../Word2vec/model/model.w2v")

C:\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [16]:
DIM = w2v_model.vector_size 
# Инициализируем матрицу embedding слоя нулями
embedding_matrix = np.zeros((NUM, DIM))
# Добавляем NUM=100000 наиболее часто встречающихся слов из обучающей выборки в embedding слой
for word, i in tokenizer.word_index.items():
    if i >= NUM:
        break
    if word in w2v_model.wv.vocab.keys():
        embedding_matrix[i] = w2v_model.wv[word]

# инициализируем слои модели

In [17]:
LayorInput = Input(shape=(SENTENCE_LENGTH,), dtype='int32')
LayorEncoded = Embedding(NUM, DIM, input_length=SENTENCE_LENGTH,
                          weights=[embedding_matrix], trainable=False)(LayorInput)

Instructions for updating:
Colocations handled automatically by placer.


# создаем CNN модель

In [21]:
branches = []
x = Dropout(rate = 0.2)(LayorEncoded)

for size, filters_count in [(2, 10), (3, 10), (4, 10), (5, 10)]:
    for i in range(filters_count):
        branch = Conv1D(filters=1, kernel_size=size, padding='valid', activation='relu')(x)
        branch = GlobalMaxPooling1D()(branch)
        branches.append(branch)

x = concatenate(branches, axis=1)
x = Dropout(rate = 0.2)(x)
x = Dense(30, activation='relu')(x)
x = Dense(1)(x)
output = Activation('sigmoid')(x)
print(filters_count)

10


In [22]:
model = Model(inputs=[LayorInput], outputs=[output])
model.compile(loss='binary_crossentropy', optimizer='adam')
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 26)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 26, 200)      20000000    input_1[0][0]                    
__________________________________________________________________________________________________
dropout_5 (Dropout)             (None, 26, 200)      0           embedding_1[0][0]                
__________________________________________________________________________________________________
conv1d_81 (Conv1D)              (None, 25, 1)        401         dropout_5[0][0]                  
__________________________________________________________________________________________________
conv1d_82 

In [ ]:
del model 

# загружаем модель

In [ ]:
model = load_model('cnn/model.h5py')

# добавим кастомные метрики

In [23]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[precision, recall, f1])

# тренировка модели

In [ ]:
checkpoint = ModelCheckpoint("cnn/cnn-frozen-embeddings-{epoch:02d}-{val_f1:.2f}.hdf5", 
                             monitor='val_f1', save_best_only=True, mode='max', period=1)
history = model.fit(x_train_seq, y_train, batch_size=32, epochs=10, validation_split=0.25, callbacks = [checkpoint])

In [ ]:
plot_history(history)

In [ ]:
model.load_weights('cnn/cnn-frozen-embeddings-05-0.77.hdf5')

In [ ]:
predicted = np.round(model.predict(x_test_seq))
print(classification_report(y_test, predicted, digits=5))

In [24]:
model.layers[1].trainable = True
adam = optimizers.Adam(lr=0.0001)
model.compile(loss='binary_crossentropy', optimizer=adam)
model.summary()
model.save('model/model_complite.h5py')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 26)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 26, 200)      20000000    input_1[0][0]                    
__________________________________________________________________________________________________
dropout_5 (Dropout)             (None, 26, 200)      0           embedding_1[0][0]                
__________________________________________________________________________________________________
conv1d_81 (Conv1D)              (None, 25, 1)        401         dropout_5[0][0]                  
__________________________________________________________________________________________________
conv1d_82 

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[precision, recall, f1])

# Тренировка модели

In [ ]:
checkpoint = ModelCheckpoint("cnn/cnn-trainable-{epoch:02d}-{val_f1:.2f}.hdf5", 
                             monitor='val_f1', save_best_only=True, mode='max', period=1)

history_trainable = model.fit(x_train_seq, y_train, batch_size=32, epochs=5, validation_split=0.25, callbacks = [checkpoint])

In [ ]:
plot_history(history_trainable)

In [ ]:
predicted = np.round(model.predict(x_test_seq))
print(classification_report(y_test, predicted, digits=5))

# загружаем обученную модель

In [ ]:
#model = load_model('cnn/model_complite.h5py')

In [ ]:
model.load_weights('cnn/cnn-trainable-03-0.78.hdf5')

In [ ]:
GetAnalis('завтра празник')